In [1]:
import pandas as pd
import re
from oso import fetch_data

In [2]:
# Global setting for whether you want grab new data or used a local archive

CONNECT_TO_OSO = False

In [3]:
# Helpers

def stringify_array(arr):
    return "'" + "','".join(arr) + "'"

def parse_github(pkg_url):
    if isinstance(pkg_url, str):
        match = re.search(r'github\.com/([^/]+)/([^/.#]+)', pkg_url)
        if match:
            owner = match.group(1).lower()
            repo = match.group(2).lower()
            return '/'.join([owner, repo])    
    return None

def extract_namespace(npm_package_name):
    if npm_package_name.startswith('@'):
        return npm_package_name.split('/')[0][1:]
    return None

# Get SBOMs for repos we care about

In [4]:
CONSENSUS = [
    'prysmaticlabs/prysm',
    'sigp/lighthouse',
    'consensys/teku',
    'status-im/nimbus-eth2',
    'chainsafe/lodestar',
    'grandinetech/grandine'
]
EXECUTION = [
    'ethereum/go-ethereum',
    'nethermindeth/nethermind',
    'hyperledger/besu',
    'erigontech/erigon',
    'paradigmxyz/reth'
]
OTHER = [
    'ethereum/solidity',
    'ethereum/remix-project',
    'vyperlang/vyper',
    'ethereum/web3.py',
    'ethereum/py-evm',
    'eth-infinitism/account-abstraction',
    'safe-global/safe-smart-account',
    'a16z/helios',
    'web3/web3.js', # prev. 'ethereum/web3.js',
    'ethereumjs/ethereumjs-monorepo'    
]
SEED_REPOS = CONSENSUS + EXECUTION + OTHER
PACKAGE_SERVERS = ['NPM', 'RUST', 'GO', 'PIP']

In [5]:
print(SEED_REPOS)

['prysmaticlabs/prysm', 'sigp/lighthouse', 'consensys/teku', 'status-im/nimbus-eth2', 'chainsafe/lodestar', 'grandinetech/grandine', 'ethereum/go-ethereum', 'nethermindeth/nethermind', 'hyperledger/besu', 'erigontech/erigon', 'paradigmxyz/reth', 'ethereum/solidity', 'ethereum/remix-project', 'vyperlang/vyper', 'ethereum/web3.py', 'ethereum/py-evm', 'eth-infinitism/account-abstraction', 'safe-global/safe-smart-account', 'a16z/helios', 'web3/web3.js', 'ethereumjs/ethereumjs-monorepo']


In [6]:
sbom_query = f"""
    WITH sboms AS (
      SELECT DISTINCT
        artifact_namespace AS repo_owner,
        CONCAT(artifact_namespace, '/', artifact_name) AS repo_name,
        package_artifact_name,
        package_artifact_source
      FROM `oso.int_sbom_artifacts`
      WHERE package_artifact_source IN ({stringify_array(PACKAGE_SERVERS)})
    )
    SELECT * FROM sboms
    WHERE repo_name IN ({stringify_array(SEED_REPOS)})
"""
df_sbom = fetch_data(sbom_query, 'data/archive/sbom.csv', connect_to_oso=CONNECT_TO_OSO)
df_sbom.tail()

,repo_owner,repo_name,package_artifact_name,package_artifact_source
13801,consensys,consensys/teku,eslint-utils,NPM
13802,consensys,consensys/teku,js-yaml,NPM
13803,consensys,consensys/teku,minimatch,NPM
13804,consensys,consensys/teku,git-url-parse,NPM
13805,consensys,consensys/teku,@babel/helper-validator-identifier,NPM


In [7]:
df_sbom.groupby('package_artifact_source')['package_artifact_name'].nunique()

package_artifact_source
GO       416
NPM     4749
PIP      137
RUST    1076
Name: package_artifact_name, dtype: int64

In [8]:
df_sbom.groupby('package_artifact_source')['package_artifact_name'].nunique().sum()

6378

## Lookup Go packages

In [9]:
df_go = df_sbom[df_sbom['package_artifact_source'] == 'GO'].copy()
df_go['package_repo_name'] = df_go['package_artifact_name'].apply(parse_github)
df_go['likely_package_repo_owner'] = df_go['package_repo_name'].apply(
    lambda x: x.split('/')[0] if isinstance(x, str) else None
)
df_go.tail()

,repo_owner,repo_name,package_artifact_name,package_artifact_source,package_repo_name,likely_package_repo_owner
13770,ethereum,ethereum/go-ethereum,github.com/kilic/bls12-381,GO,kilic/bls12-381,kilic
13797,erigontech,erigontech/erigon,github.com/garslo/gogen,GO,garslo/gogen,garslo
13798,erigontech,erigontech/erigon,github.com/mmcloughlin/addchain,GO,mmcloughlin/addchain,mmcloughlin
13799,erigontech,erigontech/erigon,github.com/libp2p/go-msgio,GO,libp2p/go-msgio,libp2p
13800,erigontech,erigontech/erigon,github.com/prometheus/client_model,GO,prometheus/client_model,prometheus


## Lookup Python packages

In [10]:
df_py = df_sbom[df_sbom['package_artifact_source'] == 'PIP'].copy()
py_pkgs_all = df_py['package_artifact_name'].dropna().unique()

In [11]:
py_pkg_lookup_query = f"""
    WITH github_data AS (
      SELECT DISTINCT
        LOWER(name) AS package_name,
        LOWER(REGEXP_EXTRACT(url, r'github\.com/([^/]+)/')) AS github_owner,
        LOWER(REGEXP_EXTRACT(url, r'github\.com/[^/]+/([^/.]+)')) AS github_repo
      FROM `bigquery-public-data.pypi.distribution_metadata`,
      UNNEST(project_urls) AS url
      WHERE url LIKE '%github.com/%'

      UNION ALL

      SELECT DISTINCT
        LOWER(name) AS package_name,
        LOWER(REGEXP_EXTRACT(home_page, r'github\.com/([^/]+)/')) AS github_owner,
        LOWER(REGEXP_EXTRACT(home_page, r'github\.com/[^/]+/([^/.]+)')) AS github_repo
      FROM `bigquery-public-data.pypi.distribution_metadata`
      WHERE home_page LIKE '%github.com/%'
    )
    
    SELECT DISTINCT
      package_name,
      github_owner,
      github_repo
    FROM github_data
    WHERE package_name IN ({stringify_array(py_pkgs_all)})
"""
df_py_pkg = fetch_data(py_pkg_lookup_query, 'data/archive/pypi_pkgs.csv', connect_to_oso=CONNECT_TO_OSO)
df_py_pkg.tail(1)

,package_name,github_owner,github_repo
179,tox,tox-dev,tox


In [12]:
df_py_pkg['package_repo_name'] = df_py_pkg.apply(
    lambda x: f"{x['github_owner']}/{x['github_repo']}".strip("'")
              if x['github_owner'] and x['github_repo']
              else None,
    axis=1
)
py_mapper = (
    df_py_pkg[['package_name', 'package_repo_name']]
    .dropna()
    .drop_duplicates()
    .set_index('package_name')['package_repo_name']
    .to_dict()
)
df_py['package_repo_name'] = df_py['package_artifact_name'].map(py_mapper)
df_py['likely_package_repo_owner'] = df_py['package_repo_name'].apply(
    lambda x: x.split('/')[0] if isinstance(x, str) else None
)

df_py.tail(1)

,repo_owner,repo_name,package_artifact_name,package_artifact_source,package_repo_name,likely_package_repo_owner
13766,status-im,status-im/nimbus-eth2,jupyter-autotime,PIP,takelushi/jupyter-autotime,takelushi


## Lookup Rust

In [13]:
df_rust = df_sbom[df_sbom['package_artifact_source'] == 'RUST'].copy()
rust_pkgs_all = df_rust['package_artifact_name'].dropna().unique()

In [14]:
rust_pkg_lookup_query = f"""
    with results as (
      select distinct
        name,
        coalesce(lower(repository), lower(homepage)) as repository
      from `crates.crates`
      where name in ({stringify_array(rust_pkgs_all)})
    )
    select * 
    from results
    where repository like '%github.com/%'
"""

df_rust_pkg = fetch_data(rust_pkg_lookup_query, 'data/archive/rust_pkgs.csv', connect_to_oso=CONNECT_TO_OSO)
df_rust_pkg['package_repo_name'] = df_rust_pkg['repository'].apply(parse_github)
df_rust_pkg.tail(1)

,name,repository,package_repo_name
1044,crossbeam-skiplist,https://github.com/crossbeam-rs/crossbeam,crossbeam-rs/crossbeam


In [15]:
df_rust = df_sbom[df_sbom['package_artifact_source'] == 'RUST'].copy()

crates_mapper = (
    df_rust_pkg[['name', 'package_repo_name']]
    .dropna()
    .drop_duplicates()
    .set_index('name')['package_repo_name']
    .to_dict()
)
df_rust['package_repo_name'] = df_rust['package_artifact_name'].map(crates_mapper)
df_rust['likely_package_repo_owner'] = df_rust['package_repo_name'].apply(
    lambda x: x.split('/')[0] if isinstance(x, str) else None
)

df_rust.tail()

,repo_owner,repo_name,package_artifact_name,package_artifact_source,package_repo_name,likely_package_repo_owner
13792,sigp,sigp/lighthouse,indenter,RUST,yaahc/indenter,yaahc
13793,sigp,sigp/lighthouse,oid-registry,RUST,rusticata/oid-registry,rusticata
13794,sigp,sigp/lighthouse,async-channel,RUST,smol-rs/async-channel,smol-rs
13795,sigp,sigp/lighthouse,ethers-core,RUST,gakonst/ethers-rs,gakonst
13796,sigp,sigp/lighthouse,anstream,RUST,rust-cli/anstyle,rust-cli


## Lookup NPM

In [16]:
df_npm = df_sbom[df_sbom['package_artifact_source'] == 'NPM'].dropna().copy()
npm_pkgs_all = sorted(df_npm['package_artifact_name'].unique())
len(npm_pkgs_all)

4749

In [17]:
# create a registry from deps.dev; note this is an expensive query

npm_pkg_lookup_query = f"""
    SELECT
      Name as package_name,
      (SELECT lower(URL) 
       FROM UNNEST(Links) 
       WHERE REGEXP_CONTAINS(URL, r'https://github\.com/')
       LIMIT 1) AS package_github_url
    FROM `bigquery-public-data.deps_dev_v1.PackageVersionsLatest`
    WHERE
      System = 'NPM'
      AND SnapshotAt >= '2024-11-01'
      AND ARRAY_LENGTH(Links) > 0
      AND lower(Name) in ({stringify_array(npm_pkgs_all)})
"""

df_npm_pkg = fetch_data(npm_pkg_lookup_query, 'data/archive/npm_pkgs.parquet', connect_to_oso=CONNECT_TO_OSO)

In [18]:
df_npm_pkg['package_repo_name'] = df_npm_pkg['package_github_url'].apply(parse_github)
npm_mapper = (
    df_npm_pkg[['package_name', 'package_repo_name']]
    .dropna()
    .drop_duplicates()
    .set_index('package_name')['package_repo_name']
    .to_dict()
)
df_npm['package_repo_name'] = df_npm['package_artifact_name'].map(npm_mapper)

df_npm['namespace'] = df_npm['package_artifact_name'].apply(extract_namespace)
namespace_to_owner = {}
for package_name, repo_name in npm_mapper.items():
    namespace = extract_namespace(package_name)
    if namespace and repo_name:
        github_owner = repo_name.split('/')[0]
        namespace_to_owner.update({namespace:github_owner})

df_npm['likely_package_repo_owner'] = df_npm['namespace'].map(namespace_to_owner)
df_npm.loc[df_npm['package_repo_name'].notna(), 'likely_package_repo_owner'] = df_npm['package_repo_name'].str.split('/').str[0]
df_npm.drop(columns='namespace', inplace=True)
df_npm.tail()

,repo_owner,repo_name,package_artifact_name,package_artifact_source,package_repo_name,likely_package_repo_owner
13801,consensys,consensys/teku,eslint-utils,NPM,mysticatea/eslint-utils,mysticatea
13802,consensys,consensys/teku,js-yaml,NPM,nodeca/js-yaml,nodeca
13803,consensys,consensys/teku,minimatch,NPM,isaacs/minimatch,isaacs
13804,consensys,consensys/teku,git-url-parse,NPM,ionicabizau/node-git-url-parse,ionicabizau
13805,consensys,consensys/teku,@babel/helper-validator-identifier,NPM,babel/babel,babel


# Consolidate back into a single graph

In [19]:
df = pd.concat([df_go,df_py,df_rust,df_npm], axis=0, ignore_index=True)
df.fillna('', inplace=True)
df.rename(columns={
    'repo_name': 'seed_repo_name',
    'repo_owner': 'seed_repo_owner',
    'package_artifact_name': 'package_name',
    'package_artifact_source': 'package_source',
    'likely_package_repo_owner': 'package_repo_owner'
}, inplace=True)
df = df[[
    'seed_repo_name', 'seed_repo_owner', 'package_name', 
    'package_repo_owner', 'package_repo_name', 'package_source'
]]
df.tail()

,seed_repo_name,seed_repo_owner,package_name,package_repo_owner,package_repo_name,package_source
13797,consensys/teku,consensys,eslint-utils,mysticatea,mysticatea/eslint-utils,NPM
13798,consensys/teku,consensys,js-yaml,nodeca,nodeca/js-yaml,NPM
13799,consensys/teku,consensys,minimatch,isaacs,isaacs/minimatch,NPM
13800,consensys/teku,consensys,git-url-parse,ionicabizau,ionicabizau/node-git-url-parse,NPM
13801,consensys/teku,consensys,@babel/helper-validator-identifier,babel,babel/babel,NPM


In [20]:
for pkg in PACKAGE_SERVERS:
    print(f"\n### Most Popular {pkg} Packages ###")
    pkg_lst = df[(df['package_source']==pkg) & (df['package_repo_owner'] != '')]['package_repo_name'].value_counts()
    nth = int(len(pkg_lst) * (0.025 if pkg == 'NPM' else 0.1))
    pkg_lst_top = list(pkg_lst.head(nth).index)
    print(pkg_lst_top)


### Most Popular NPM Packages ###
['babel/babel', 'definitelytyped/definitelytyped', 'ethers-io/ethers', 'lerna/lerna', 'lodash/lodash', 'ben-eb/cssnano', 'ethereumjs/ethereumjs-monorepo', 'evanw/esbuild', 'micromark/micromark', '', 'chainsafe/web3', 'streetsidesoftware/cspell-dicts', 'facebook/jest', 'facebook/docusaurus', 'xtuc/webassemblyjs', 'jestjs/jest', 'gregberge/svgr', 'radix-ui/primitives', 'typescript-eslint/typescript-eslint', 'rollup/rollup', 'blakeembrey/change-case', 'nrwl/nx', 'tootallnate/proxy-agents', 'nomicfoundation/solidity-analyzer', 'getsentry/sentry-javascript', 'napi-rs/node-rs', 'ethereum/web3', 'istanbuljs/istanbuljs', 'algolia/algoliasearch-client-javascript', 'swc-project/swc', 'achingbrain/it', 'ardatan/graphql-tools', 'tsconfig/bases', 'eslint/js', 'streetsidesoftware/cspell', 'vitest-dev/vitest', 'facebook/react', 'nodelib/nodelib', 'jaredwray/cacheable', 'stablelib/stablelib', 'es-shims/string', 'webdriverio/webdriverio', 'walletconnect/walletconnect-

In [21]:
owners = list(df['package_repo_owner'].unique())
len(owners)

1735

In [22]:
df.to_csv('data/unweighted_graph.csv')

# Create a network graph

In [23]:
import json
import networkx as nx

In [24]:
dff = df[df['package_repo_owner'] != ''].copy()
len(dff) / len(df)

0.9802202579336328

In [25]:
gh = 'https://github.com/'
dff['source'] = dff['seed_repo_name'].apply(lambda x: f'{gh}{x}')
dff['target'] = dff.apply(
    lambda x: f"{gh}{x['package_repo_name']}"
              if x['package_repo_name'] != ''
              else f"{gh}{x['package_repo_owner']}"
    , axis=1)
dff

,seed_repo_name,seed_repo_owner,package_name,package_repo_owner,package_repo_name,package_source,source,target
0,prysmaticlabs/prysm,prysmaticlabs,github.com/mattn/go-runewidth,mattn,mattn/go-runewidth,GO,https://github.com/prysmaticlabs/prysm,https://github.com/mattn/go-runewidth
2,prysmaticlabs/prysm,prysmaticlabs,github.com/minio/highwayhash,minio,minio/highwayhash,GO,https://github.com/prysmaticlabs/prysm,https://github.com/minio/highwayhash
3,prysmaticlabs/prysm,prysmaticlabs,github.com/leodido/go-urn,leodido,leodido/go-urn,GO,https://github.com/prysmaticlabs/prysm,https://github.com/leodido/go-urn
4,prysmaticlabs/prysm,prysmaticlabs,github.com/juju/ansiterm,juju,juju/ansiterm,GO,https://github.com/prysmaticlabs/prysm,https://github.com/juju/ansiterm
5,prysmaticlabs/prysm,prysmaticlabs,github.com/go-sourcemap/sourcemap,go-sourcemap,go-sourcemap/sourcemap,GO,https://github.com/prysmaticlabs/prysm,https://github.com/go-sourcemap/sourcemap
...,...,...,...,...,...,...,...,...
13797,consensys/teku,consensys,eslint-utils,mysticatea,mysticatea/eslint-utils,NPM,https://github.com/consensys/teku,https://github.com/mysticatea/eslint-utils
13798,consensys/teku,consensys,js-yaml,nodeca,nodeca/js-yaml,NPM,https://github.com/consensys/teku,https://github.com/nodeca/js-yaml
13799,consensys/teku,consensys,minimatch,isaacs,isaacs/minimatch,NPM,https://github.com/consensys/teku,https://github.com/isaacs/minimatch
13800,consensys/teku,consensys,git-url-parse,ionicabizau,ionicabizau/node-git-url-parse,NPM,https://github.com/consensys/teku,https://github.com/ionicabizau/node-git-url-parse


In [26]:
G = nx.DiGraph()

for repo in dff['seed_repo_name'].unique():
    repo_url = f"{gh}{repo}"
    G.add_node(repo_url, level=1)
    
for repo in dff['package_repo_name'].unique():
    repo_url = f"{gh}{repo}"
    if repo_url not in G.nodes:
        G.add_node(repo_url, level=2)
        
for _, row in dff.iterrows():
    G.add_edge(row['source'], row['target'], relation=row['package_source'])
    
total_edges = G.number_of_edges()
print(total_edges)

global_weight = 0
for u, v in G.edges:
    G[u][v]['weight'] = global_weight
    
graph_json = nx.node_link_data(G)
output_path = "data/unweighted_graph.json"
with open(output_path, "w") as f:
    json.dump(graph_json, f, indent=2)

9873
